<a href="https://colab.research.google.com/github/vhuni/hackathon_chatbot/blob/main/akero_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install rasa

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 838.7/838.7 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.5/203.5 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 70.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 45.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
!pip install nest_asyncio

In [3]:
import os
import rasa
import nest_asyncio

nest_asyncio.apply()

print("event loop is ready")

event loop is ready


In [4]:
from rasa.cli.scaffold import create_initial_project

project = 'akero_chatbot'
create_initial_project(project)

In [5]:
os.chdir(project)
print(os.listdir("."))

['endpoints.yml', 'data', 'tests', 'actions', 'domain.yml', 'credentials.yml', 'config.yml']


In [6]:
config = "config.yml"
training_files = "data/"
domain = "domain.yml"
output = "models/"


In [101]:
%%writefile config.yml

recipe: default.v1
assistant_id: example_bot
language: en

pipeline:
# will be selected by the Suggested Config feature
# - name: FallbackClassifier
#   threshold: 0.5

policies:
- name: MemoizationPolicy
- name: TEDPolicy
  max_history: 5
  epochs: 10

# - name: RulePolicy
#   # Confidence threshold for the `core_fallback_action_name` to apply.
#   # The action will apply if no other action was predicted with
#   # a confidence >= core_fallback_threshold
#   core_fallback_threshold: 0.3
#   core_fallback_action_name: "action_default_fallback"
#   enable_fallback_prediction: True



Overwriting config.yml


In [95]:
%%writefile actions.py

from typing import Any, Text, Dict, List

from rasa_sdk import Action, Tracker
from rasa_sdk.events import UserUtteranceReverted
from rasa_sdk.executor import CollectingDispatcher

class ActionDefaultFallback(Action):
    """Executes the fallback action and goes back to the previous state
    of the dialogue"""

    def name(self) -> Text:
        return utter_default

    async def run(
        self,
        dispatcher: CollectingDispatcher,
        tracker: Tracker,
        domain: Dict[Text, Any],
    ) -> List[Dict[Text, Any]]:
        dispatcher.utter_message(template="my_custom_fallback_template")

        # Revert user message which led to fallback.
        return [UserUtteranceReverted()]

Writing actions.py


In [114]:
%%writefile domain.yml


intents:
  - greet
  - akero_inquiry
  - campaign_inquiry
  - tactic_inquiry
  - affirm
  - goodbye


actions:
  - utter_greet
  - utter_akero_req
  - utter_tactic_req
  - utter_campaign_req
  - utter_affirm
  - __main__.ApiAction
  - utter_goodbye
  - utter_welcome
  # - utter_default

responses:
  utter_greet:
  - text: "Hi how you doing"

  utter_akero_req:
  - text: "The student advertising and conversion platform powered by AI supported by Higher Education experts."

  utter_tactic_req:
  - text: "Advertising tactics are strategic actions that promote a product or service. In Akero your tactics will be from different online platforms platforms such as Google, Facebook, LinkedIn or even Spotify! Click on the link [tactics](https://drive.google.com/file/d/1naOLgdix5ZKtbMK_A2KDTiQKwgSYOGd7/view?usp=sharing)"

  utter_campaign_req:
  - text: "Marketing campaigns are the overall view of different tactics. Click on the [campaigns](https://drive.google.com/file/d/1j5w0DakAMjIGnNM77NP5-xzbHxnHRo9R/view?usp=sharing) to see an example"

  utter_affirm:
  - text: "You're welcome"

  utter_welcome:
  - text: "Have a great day!"

  # utter_default:
  # - text: "Sorry, but I don't understand your question. Can you rephrase your question? For more information please check [confluence](https://akerolabs.atlassian.net/wiki/spaces/BT/overview?homepageId=491588)"

session_config:
  session_expiration_time: 60  # value in minutes
  carry_over_slots_to_new_session: true

Overwriting domain.yml


In [115]:
%%writefile data/nlu.yml

version: "3.1"

nlu:

# - intent: welcome
#   examples: |
#     - Welcome to Akero
#     - how can I help you today?
#     - It's good to see you


- intent: greet
  examples: |
    - hey
    - hello there
    - hi
    - hello
    - good morning
    - good evening
    - hey there
    - let's go
    - goodmorning
    - goodevening
    - good afternoon

- intent: affirm
  examples: |
    - great
    - OK
    - okay
    - thank you
    - thanks
    - yes, thanks
    - cheers
    - thanks a lot
    - thank you very much

- intent: akero_inquiry
  examples: |
    - What is akero ?
    - Is akero a marketing application
    - Is akero created by NetNatives
    - akero

- intent: campaign_inquiry
  examples: |
    - What are campaigns?
    - Could you explain what campaigns are?
    - campaigns
    - projects

- intent: tactic_inquiry
  examples: |
    - Who creates tactics?
    - what are tactics?
    - tactics
    - facebook
    - google

- intent: goodbye
  examples: |
    - bye
    - goodbye

Overwriting data/nlu.yml


In [ ]:
# print(config, training_files, domain, output)

In [104]:
%%writefile data/rules.yml

version: "3.1"

# rules:

# - rule: Say goodbye anytime the user says goodbye
#   steps:
#   - intent: goodbye
#   - action: utter_welcome

# - rule: say bye
#   steps:
#   - intent: affirm
#   - action: utter_welcome

# - rule: welcome from akero
#   steps:
#   - intent: greet
#   - action: utter_welcome

# - rule: can't understand question
#   steps:
#   - intent: nlu_fallback
#   - action: utter_default


Overwriting data/rules.yml


In [116]:
%%writefile data/stories.yml

## path 1
stories:
- story : akero_inq_1
  steps:
  - intent: greet
  - action: utter_greet
  - intent : akero_inquiry
  - action : utter_akero_req
  # - intent : affirm
  # - action : utter_affirm

- story : campaign_inq_1
  steps:
  - intent: greet
  - action: utter_greet
  - intent : campaign_inquiry
  - action : utter_campaign_req
  # - intent : affirm
  # - action : utter_affirm

- story: tactic_inq_1
  steps:
  - intent: greet
  - action: utter_greet
  - intent: tactic_inquiry
  - action: utter_tactic_req
  # - intent: affirm
  # - action: utter_affirm


# - story: greet_1
#   steps:
#   - intent: greet
#   - action: utter_greet

Overwriting data/stories.yml


In [117]:
## Training the data

model_path = rasa.train(domain, config, [training_files], output)
print(model_path)

/usr/local/lib/python3.10/dist-packages/rasa/shared/utils/io.py:99: UserWarning: Action 'utter_goodbye' is listed as a response action in the domain file, but there is no matching response defined. Please check your domain.
  More info at https://rasa.com/docs/rasa/responses
/usr/local/lib/python3.10/dist-packages/rasa/shared/utils/io.py:99: UserWarning: 'RulePolicy' is not included in the model's policy configuration. Default intents such as 'restart' and 'back' will not trigger actions 'action_restart' and 'action_back'.
  More info at https://rasa.com/docs/rasa/default-actions


The configuration for pipeline was chosen automatically. It was written into the config file at 'config.yml'.


Epochs: 100%|██████████| 100/100 [00:24<00:00,  4.07it/s, t_loss=0.988, i_acc=1]


Your Rasa model is trained and saved at 'models/20240419-104005-hard-guaranty.tar.gz'.
TrainingResult(model='models/20240419-104005-hard-guaranty.tar.gz', code=0, dry_run_results=None)


<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead


In [53]:
!rasa run actions & rasa shell

/usr/local/lib/python3.10/dist-packages/rasa/core/tracker_store.py:1044: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base: DeclarativeMeta = declarative_base()
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:121: DeprecationWarning: pkg_resources is deprecated as an API
  warnings.warn("pkg_resources is deprecated as an API", DeprecationWarning)
/usr/local/lib/python3.10/dist-packages/rasa/core/tracker_store.py:1044: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades pr

In [119]:
from rasa.jupyter import chat

endpoints = None
chat(model_path.model, endpoints)

Your bot is ready to talk! Type your messages here or send '/stop'.
hello
2024-04-19 10:42:59 [debug    ] processor.actions.policy_prediction prediction_events=[]
2024-04-19 10:42:59 [debug    ] processor.actions.log          action_name=action_session_start rasa_events=[<rasa.shared.core.events.SessionStarted object at 0x7bb83963bac0>, ActionExecuted(action: action_listen, policy: None, confidence: None)]
2024-04-19 10:42:59 [debug    ] processor.slots.log            slot_values=	session_started_metadata: None
2024-04-19 10:43:00 [debug    ] processor.message.parse        parse_data_entities=[] parse_data_intent={'name': 'greet', 'confidence': 0.9999998807907104} parse_data_text=hello
2024-04-19 10:43:00 [debug    ] processor.extract.slots        action_extract_slot=action_extract_slots len_extraction_events=0 rasa_events=[]
2024-04-19 10:43:00 [debug    ] memoization.predict.actions    tracker_states=[{}, {'user': {'intent': 'greet'}, 'prev_action': {'action_name': 'action_listen'}}]

Have a great day!
Marketing campaigns are the overall view of different tactics. Click on the [campaigns](https://drive.google.com/file/d/1j5w0DakAMjIGnNM77NP5-xzbHxnHRo9R/view?usp=sharing) to see an example
Marketing campaigns are the overall view of different tactics. Click on the [campaigns](https://drive.google.com/file/d/1j5w0DakAMjIGnNM77NP5-xzbHxnHRo9R/view?usp=sharing) to see an example
Marketing campaigns are the overall view of different tactics. Click on the [campaigns](https://drive.google.com/file/d/1j5w0DakAMjIGnNM77NP5-xzbHxnHRo9R/view?usp=sharing) to see an example
Marketing campaigns are the overall view of different tactics. Click on the [campaigns](https://drive.google.com/file/d/1j5w0DakAMjIGnNM77NP5-xzbHxnHRo9R/view?usp=sharing) to see an example
Marketing campaigns are the overall view of different tactics. Click on the [campaigns](https://drive.google.com/file/d/1j5w0DakAMjIGnNM77NP5-xzbHxnHRo9R/view?usp=sharing) to see an example
Marketing campaigns are the over

hi
2024-04-19 10:44:34 [debug    ] processor.message.parse        parse_data_entities=[] parse_data_intent={'name': 'greet', 'confidence': 0.9999998807907104} parse_data_text=hi
2024-04-19 10:44:34 [debug    ] processor.extract.slots        action_extract_slot=action_extract_slots len_extraction_events=0 rasa_events=[]


/stop
